### Collecting stats from isoquant outputs

Collecting the SQANTI-style classification calls from the IsoQuant outputs.

### imports

In [ ]:
import gzip
import pickle
from collections import defaultdict, Counter
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path

import os
import numpy as np
import pysam
import yaml

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import mdl.sc_isoform_paper.plots  # noqa
from mdl.sc_isoform_paper.constants import MASSEQ_KEYS, SAMPLE_COLORS, SHORTREAD_KEYS
from mdl.sc_isoform_paper.pipseq_barcodes import barcode_to_sequence

from mdl.isoscelles.leiden import cluster_leaf_nodes, cluster_labels

### Setup

In [ ]:
root_dir = Path.home()
data_path = root_dir / "data"

masseq_data_path = data_path / "masseq"
annotated_path = masseq_data_path / "20250124_annotated"

sqanti_stats_file = data_path / "isoquant_sqanti3_classes.pickle"

figure_path = root_dir / "202501_figures"

In [ ]:
sample_order = [MASSEQ_KEYS[i] for i in (1, 3, 4)]
sample_order

In [ ]:
if sqanti_stats_file.exists():
    with sqanti_stats_file.open("rb") as fh:
        c_counters = pickle.load(fh)

## Read SQANTI classifications

If you have the pickle file, you can skip to **Plotting and tables**.

Otherwise we need to read the clustering from notebook 01 and then count the SQANTI3 classifications (from the IsoQuant output) broken down by cluster assignment.

In [ ]:
annotated_bams = sorted(annotated_path.glob("*.[134].*annotated.bam"))
len(annotated_bams)

In [ ]:
with open(data_path / "shortread_clustering_100k.pickle", "rb") as fh:
    sr_clustering = pickle.load(fh)

with open(data_path / "shortread_stats_100k.pickle", "rb") as fh:
    ix_dict = pickle.load(fh)["ix_dict"]

with open(data_path / "shortread_stats_100k.pickle", "rb") as fh:
    sr_numis = pickle.load(fh)["numis"]

In [ ]:
c_arrays = dict()
for key in SHORTREAD_KEYS:
    _leaf_nodes = cluster_leaf_nodes(sr_clustering[key][0])
    _label_array = cluster_labels(sr_clustering[key][0], _leaf_nodes)
    _k2i = {k: i for i, k in enumerate(sorted(_leaf_nodes))}    
    c_arrays[key] = np.array([_k2i.get(k, -1) for k in _label_array])

In [ ]:
bc_dict = dict()

for fp in data_path.glob("10x*/outs/raw_feature_bc_matrix/barcodes.tsv.gz"):
    fp = fp.parent
    print(fp)
    with gzip.open(fp / "barcodes.tsv.gz", "rt") as fh:
        # strip off the -1 suffix from barcodes
        bc_dict[fp.parent.parent.name] = [line.strip()[:-2] for line in fh]

fp = data_path / "pipseq_pbmc"
print(fp)
with gzip.open(fp / "raw_matrix" / "barcodes.tsv.gz", "rt") as fh:
    # convert from encoded barcode to the real sequence
    bc_dict[fp.name] = [barcode_to_sequence(line.strip()) for line in fh]


In [ ]:
def read_class_counts_w_bc(key, bam_file, bc_dict):
    sq_class_counts = defaultdict(Counter)

    with pysam.AlignmentFile(bam_file, "rb", threads=2) as fh:
        for a in fh:
            bc = a.get_tag("CB")
            yc = a.get_tag("XS")
            if bc in bc_dict:
                sq_class_counts[bc_dict[bc]][yc] += 1

    return key, dict(sq_class_counts)

In [ ]:
bc_to_cluster = {
    k: dict(
        zip(
            (bc for bc,i in zip(bc_dict[k2], ix_dict[k2]) if i),
            c_arrays[k2]
        )
    ) for k, k2 in zip(sample_order, ["pipseq_pbmc", "10x_3p_pbmc", "10x_5p_pbmc"])
}

In [ ]:
key_list = [MASSEQ_KEYS[int(ab.name.split(".")[2])] for ab in annotated_bams]

with ProcessPoolExecutor(8) as exc:
    c_counters = defaultdict(lambda: defaultdict(Counter))
    for key, cc in exc.map(
        read_class_counts_w_bc, 
        key_list,
        annotated_bams,
        (bc_to_cluster[key] for key in key_list)
    ):
        for i in cc:
            c_counters[key][i] += cc[i]

In [ ]:
c_counters = {k: {j: v for j,v in c_counters[k].items()} for k in c_counters}

if not sqanti_stats_file.exists():
    with sqanti_stats_file.open("wb") as out:
        pickle.dump(c_counters, out)

## Plotting and tables

In [ ]:
# order categories by overall counts
c_order = [
    c for c,_ in sum((sum(c_counters[k].values(), start=Counter()) for k in c_counters), start=Counter()).most_common()
    if c != "-"
] + ["-"]

In [ ]:
# same as the labels from notebook 01
cluster_names = {
    ("PIPseq", "0.8x"): {
        0: 'CD4 T cells 1',
        1: 'CD4 T cells 2',
        2: 'Naïve CD4',
        3: 'Cytotoxic T cells',
        4: 'Innate Lymphoid',
        5: 'CD16 Monocytes',
        6: 'CD14 Monocytes',
        7: 'B cells',
    },
    ("10x 3'",): {
        0: 'CD4 T cells 1',
        1: 'CD4 T cells 2',
        2: 'Naïve CD4',
        3: 'Cytotoxic T cells',
        4: 'B cells',
        5: 'CD14 Monocytes',
        6: 'CD16 Monocytes',
        7: 'DC',
    },
    ("10x 5'",): {
        0: 'CD4 T cells 1',
        1: 'CD4 T cells 2',
        2: 'Naïve CD4',
        3: 'Cytotoxic T cells',
        4: 'Innate Lymphoid',
        5: 'B cells',
        6: 'CD14 Monocytes',
        7: 'CD16 Monocytes',
    }
}

cluster_reverse_labels = {
    k: {v: i for i,v in cluster_names[k].items()}
    for k in cluster_names
}


In [ ]:
all_cell_labels = sorted(set.union(*(set(v.values()) for v in cluster_names.values())))

x = np.arange(len(c_order))
fig, axs = plt.subplots(3, 3, figsize=(18, 9), sharex=True)

for lbl, ax in zip(all_cell_labels, axs.flatten()):
    for i, k in enumerate(c_counters):
        if lbl not in cluster_reverse_labels[k]:
            continue
            
        j = cluster_reverse_labels[k][lbl]
        y = [c_counters[k][j][c] / c_counters[k][j].total() for c in c_order]
        ax.bar(x + 0.05 + i * 0.3, width=0.3, height=y, color=SAMPLE_COLORS[k[0]], align="edge")
    ax.set_title(lbl)
    ax.set_xticks(x + 0.5, c_order, rotation=75, ha="right")
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=0))

plt.savefig(figure_path / "supp_fig7.svg")
plt.show()


In [ ]:
combined_c_counters = {k: sum(c_counters[k].values(), start=Counter()) for k in c_counters}

# supplemental table 4
print(f"{'class':23}", *(f"{' '.join(k):>16}" for k in c_counters), sep="\t")
for c in c_order:
    print(f"{c:23}", *(f"{combined_c_counters[k][c] / combined_c_counters[k].total():16.2%}" for k in combined_c_counters), sep="\t")


In [ ]:
# note: the cluster orders are not the same. We reordered the columns for supplementary table 5
for k in c_counters:
    print(f"{' '.join(k):23}", *(f"{cluster_names[k][i].replace(' ', '_'):>16}" for i in range(8)), sep="\t")
    for c in c_order:
        print(f"{c:23}", *(f"{c_counters[k][i][c] / c_counters[k][i].total():16.2%}" for i in range(8)), sep="\t")
    print()

# Analysis of downstream impact on DE

We will now take a look at how filtering for only good priming reads or not affects Differential Expression analysis when comparing cell type clusters. 

In [ ]:
from mdl.sc_isoform_paper.isoform_counts import (
    build_cell_by_isoform_matrix,
    cell_isoform_set_to_df,
    make_anndata_from_counts,
    library_prefix,
    write_h5ad,
    export_10x_from_h5ad,
)


In [ ]:
merged_annotated_path = annotated_path / "merged"
de_path = data_path / "DE_analysis"

In [ ]:
annotated_bams = {
    ("10x 3'",): bams_path / "10x_3p.primary.annotated.dedup.sorted.bam",
    ("10x 5'",): bams_path / "10x_5p.primary.annotated.dedup.sorted.bam",
    # ("PIPseq 0.6x"): bams_path / "pipseq_6x.primary.annotated.dedup.sorted.bam",
    ('PIPseq', '0.8x'): bams_path / "pipseq_8x.primary.annotated..dedup.sorted.bam",
}

In [ ]:
good_priming_categories = set(["ANNO_GPA", "GOOD", "TX_PAS", "TX_MOTIF"])

We will read the assigned isoform ID from the XI tag of our annotated, deduplicated BAMs, and create an AnnData object for both all of our reads, and only the good priming reads.

In [ ]:
# XI (annotated)
for library, bam_file in annotated_bams.items():
    cell_isoform_umis, cell_goodprim_isoform_umis = build_cell_by_isoform_matrix(
        bam_file, bc_to_cluster[library], good_priming_categories, isoform_tag="XI", threads=12
    )
    df_all  = cell_isoform_set_to_df(cell_isoform_umis)
    df_good = cell_isoform_set_to_df(cell_goodprim_isoform_umis)

    ad_all  = make_anndata_from_counts(df_all,  bc_to_cluster[library])
    ad_good = make_anndata_from_counts(df_good, bc_to_cluster[library])

    lp = library_prefix(library)
    write_h5ad(ad_all,  de_path / f"{lp}allisoform_dedup_counts.h5ad")
    write_h5ad(ad_good, de_path / f"{lp}goodpriming_dedup_counts.h5ad")


We then do the same but using the isoform IDs when also running reconstruction.

In [ ]:
# YT (reconstructed)
for library, bam_file in annotated_bams.items():
    cell_isoform_umis, cell_goodprim_isoform_umis = build_cell_by_isoform_matrix(
        bam_file, bc_to_cluster[library], good_priming_categories, isoform_tag="YT", threads=12
    )
    df_all  = cell_isoform_set_to_df(cell_isoform_umis)
    df_good = cell_isoform_set_to_df(cell_goodprim_isoform_umis)

    ad_all  = make_anndata_from_counts(df_all,  bc_to_cluster[library])
    ad_good = make_anndata_from_counts(df_good, bc_to_cluster[library])

    lp = library_prefix(library)
    write_h5ad(ad_all,  de_path / f"{lp}allisoform_reconstructed_dedup_counts.h5ad")
    write_h5ad(ad_good, de_path / f"{lp}goodpriming_reconstructed_dedup_counts.h5ad")


We will also export these count matrices in the 10X format to easily read in Seurat where we will run the Different Expression analysis.

In [ ]:
# 10X export
for library in annotated_bams.keys():
    lp = library_prefix(library)
    for filtering_type in (
        "allisoform_dedup",
        "goodpriming_dedup",
        "allisoform_reconstructed_dedup",
        "goodpriming_reconstructed_dedup",
    ):
        in_h5ad = de_path / f"{lp}{filtering_type}_counts.h5ad"
        out_dir = de_path / f"{lp}{filtering_type}"
        export_10x_from_h5ad(in_h5ad, out_dir)

Now we create the R scripts that will be used to run the DE analysis

In [ ]:
# Cluster label maps per library
labels_3p = {
    "0": "CD4 T cells 1",
    "1": "CD4 T cells 2",
    "2": "Naive CD4",
    "3": "Cytotoxic T cells",
    "4": "B cells",
    "5": "CD14 Monocytes",
    "6": "CD16 Monocytes",
    "7": "DC",
}
labels_5p = {
    "0": "CD4 T cells 1",
    "1": "CD4 T cells 2",
    "2": "Naive CD4",
    "3": "Cytotoxic T cells",
    "4": "Innate Lymphoid",
    "5": "B cells",
    "6": "CD14 Monocytes",
    "7": "CD16 Monocytes",
}
labels_pip = {
    "0": "CD4 T cells 1",
    "1": "CD4 T cells 2",
    "2": "Naive CD4",
    "3": "Cytotoxic T cells",
    "4": "Innate Lymphoid",
    "5": "CD16 Monocytes",
    "6": "CD14 Monocytes",
    "7": "B cells",
}

# Library prefixes (dir tag, sample prefix, labels)
techs = [
    ("10x_3'",  "10X_3p",         labels_3p),
    ("10x_5'",  "10X_5p",         labels_5p),
    ("PIPseq_0.8x", "PIPseq_0.8x", labels_pip),
]

# Two reconstructed variants to include
filters = [
    ("goodpriming_reconstructed_dedup", "reconstructed_goodpriming_dedup"),
    ("allisoform_reconstructed_dedup",  "reconstructed_allisoform_dedup"),
]

datasets = []
for dir_tag, sample_prefix, labels in techs:
    for dir_filter, sample_suffix in filters:
        data_dir = (de_path / f"{dir_tag}_{dir_filter}").as_posix() + "/"
        sample_name = f"{sample_prefix}_{sample_suffix}"
        output_prefix = (de_path / f"{sample_name}.isoforms_seurat").as_posix()
        datasets.append({
            "sample_name": sample_name,
            "data_dir": data_dir,
            "output_prefix": output_prefix,
            "cluster_labels": {str(k): v for k, v in labels.items()},
        })

with open(de_path / "seurat_datasets.yaml", "w") as fh:
    yaml.safe_dump({"datasets": datasets}, fh, sort_keys=False, default_flow_style=False)

print(f"Wrote seurat_datasets.yaml with {len(datasets)} entries at {Path.cwd()}")


In [ ]:
r_script = r"""#!/usr/bin/env Rscript
args_all <- commandArgs(trailingOnly = FALSE)
script_arg <- args_all[grepl("^--file=", args_all)]
this_dir <- if (length(script_arg)) dirname(normalizePath(sub("^--file=", "", script_arg))) else getwd()

args <- commandArgs(trailingOnly = TRUE)
cfg_file <- if (length(args) >= 1) args[1] else file.path(this_dir, "seurat_datasets.yaml")
template <- file.path(this_dir, "seurat_template.Rmd")

suppressPackageStartupMessages({
  library(yaml)
  library(rmarkdown)
})

cfg <- yaml::read_yaml(cfg_file)

for (d in cfg$datasets) {
  message("Rendering: ", d$sample_name)
  rmarkdown::render(
    input       = template,
    params      = list(
      sample_name   = d$sample_name,
      data_dir      = d$data_dir,
      output_prefix = d$output_prefix,
      cluster_labels= d$cluster_labels
    ),
    output_file = paste0(d$sample_name, ".md"),
    output_dir  = dirname(d$output_prefix)
  )
}
"""

out = de_path / "run_all_seurat_DE.R"
out.write_text(r_script)
os.chmod(out, os.stat(out).st_mode | 0o111)
print(f"Wrote {out.resolve()}  (looks for seurat_template.Rmd and seurat_datasets.yaml in the same {de_path} folder)")

And import the Rmd template included in the package to avoid having to search for it.

In [ ]:
from importlib.resources import files

dst = de_path/ "seurat_template.Rmd"
with files("mdl.sc_isoform_paper.templates").joinpath("seurat_template.Rmd").open("r", encoding="utf-8") as src:
    dst.write_text(src.read(), encoding="utf-8")
print(f"Copied {dst.resolve()}")


Once these matrices have been generated, we can run all Seurat DE analysis by running, from the DE_analysis directory:

./run_all_seurat_DE.R

Once the script has run, you can go to notebook 10_p2, which is an R notebook.